## __Text mining y Procesamiento de Lenguaje Natural (NLP)__

__Profesor__: Anthony D. Cho

__Tema__: Clustering de documentos

__Método__: K-Means

***

__Dependencias__

```{python}
    python -m pip install nltk spacy simpsom
    python -m spacy download en_core_web_sm
    python -m spacy download es_core_news_sm
```

## Librerias

In [ ]:
import re
from glob import glob
import matplotlib.pyplot as plt
from pandas import DataFrame
from tqdm import tqdm

from string import punctuation
from spacy.lang.es.stop_words import STOP_WORDS
from spacy import load

from sklearn.feature_extraction.text import TfidfVectorizer

from simpsom import SOMNet

## Instancia del modelo de lenguaje
nlp = load('es_core_news_sm')

## Carga de documentos

In [ ]:
## Encontrar la ruta de cada archivo de interes
path_docs = glob('*/doc*.txt')

## Almacenamiendo de contenido de los documentos e id (nombre del archivo)
corpus, doc_id = [], [] 

## Incio de proceso de carga de documentos
if len(path_docs):
    for file in path_docs:

        ## Se carga el texto
        text = open(file, 'r', encoding='utf-8').read()
        
        ## Se almacena el texto
        corpus.append(text)
        
        id = file.split('\\')[-1].split('.')[0]

        ## Se almacena el id
        doc_id.append(id)
else:
    print('No corpus have found.')

In [ ]:
doc_id

#### Preprocesamiento

In [ ]:
## Limpieza de textos
cleanTexts = []

for doc in corpus:

    # ## Remover numeros y puntuaciones
    doc = re.sub(r'[\"\¿\°\d+]', '', doc)
    doc = [s for s in doc if s not in punctuation]
    doc = ''.join(doc)

    ## Normalización y remover stopwords
    documento = nlp(doc.lower())
    tokens = [word.text for word in documento]
    doc = [word for word in tokens if word not in STOP_WORDS]
    doc = ' '.join(doc)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)
    
    ## Aplicar lemmatización
    documento = nlp(doc)
    lemmas = [word.lemma_ for word in documento]
    doc = ' '.join(lemmas)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)

    ## Almacenado de contenido procesado
    cleanTexts.append(doc)

## Mostar contenido procesado
cleanTexts
    

In [ ]:
## Instancia del modelo
model = TfidfVectorizer(use_idf=True)

## Ajuste del modelo y retorno de TF matrix como (docs, term)
tf_sparse = model.fit_transform(cleanTexts)

## Extraer la lista de los palabras que representan las columnas de la matriz generada
vocabulary = model.get_feature_names_out()

print('(shape) TFxIDF (terms, docs): {}'.format(tf_sparse.shape))

#### SOM

In [ ]:
## Instancia del modelo
model = SOMNet(net_height=10, net_width=10, 
               data=tf_sparse.toarray(), 
               random_seed=20221022)

## Ajuste del modelo
model.train(start_learning_rate=0.01, epochs=1000)

## Guardar el modelo ajustado
#model.save('SOM_simpsom')

In [ ]:
## Weights difference on SOM grid mapping display
model.diff_graph(show=True)

In [ ]:
## Asignar las etiquetas mediante el modelo ajustado
clusters = model.cluster(array=tf_sparse.toarray(), show=True)
print('Cantidad de clusteres:', len(clusters))